# BigBIRD Preprocessing

This notebook manly does two things: (a) it computes new object masks using [ClipDrop.co](https://clipdrop.co/) and (b) convert the data into the format expected by our intrinsic neural fields code. Step (a) is necessary because the object masks in the original dataset are not good enough.

**Important: It is not necessary to run this notebook or use clipdrop because weprovide all data for download. This notebook is only necessary if you want to process further classes from the BigBIRD dataset**.

In [ ]:
import os
from os.path import join, exists, dirname, basename
from glob import glob
import shutil
import matplotlib.pyplot as plt
from bigbird import *
import pickle

from tqdm.notebook import tqdm

### Data Entry

In [ ]:
bigbird_dir = "/path/to/directory/for/bigbird/data"
ddir = "/path/to/your/downloads/folder"  # this is used to copy the downloads from clipdrop

raw_dir = join(bigbird_dir, "raw")
clip_dir = join(bigbird_dir, "clipdrop_input")
clip_out_dir = join(bigbird_dir, "clipdrop_output")
proc_dir = join(bigbird_dir, "processed")

names ="""
3m_high_tack_spray_adhesive
advil_liqui_gels
band_aid_clear_strips
band_aid_sheer_strips
blue_clover_baby_toy
bumblebee_albacore
campbells_chicken_noodle_soup
campbells_soup_at_hand_creamy_tomato
canon_ack_e10_box
cheez_it_white_cheddar
"""
names = names.split()
assert len(names) == 10

In [ ]:
from fractions import Fraction
class BBox:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        assert self.x[0] >= 0
        assert self.y[0] >= 0
        
    def __repr__(self):
        return f"BBox of size {self.x[1]-self.x[0]} x {self.y[1]-self.y[0]}"
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def size(self):
        return (self.x[1]-self.x[0], self.y[1]-self.y[0])
    
    def cut(self, img):
        return img[self.y[0]:self.y[1], self.x[0]:self.x[1]]
    
    def aspect(self):
        return Fraction(self.size()[0], self.size()[1])
    
def bbox_from_mask(mask, border=30):
    assert mask.dtype == np.uint8 and mask.max() == 1
    
    xmin = np.where(np.sum(mask, 0))[0][0]-border
    xmax = np.where(np.sum(mask, 0))[0][-1]+border
    
    ymin = np.where(np.sum(mask, 1))[0][0]-border
    ymax = np.where(np.sum(mask, 1))[0][-1]+border
    
    return BBox(x=(xmin, xmax), y=(ymin, ymax))


def bbox_plot(ax, bbox, c='r'):
    xs = bbox.x + bbox.x[::-1] + bbox.x[:1]
    ys = bbox.y[:1]*2 + bbox.y[1:]*2 + bbox.y[:1]
    
    return ax.plot(xs, ys, c)

def bbox_cut_image(bbox, img):
    cut = img[bbox.y[0]:bbox.y[1], bbox.x[0]:bbox.x[1]]
    return cut
    

def plot_img_mask(img, mask, supertitle=None, bbox=None):
    fig, axs=plt.subplots(1, 3, sharex=True, sharey=True, figsize=(20,10))

    ax=axs[0]
    cm = ax.imshow(img[..., ::-1])
    plt.colorbar(cm, ax=ax)
    ax.set_title("Image")

    ax=axs[1]
    cm = ax.imshow(mask)
    plt.colorbar(cm, ax=ax)
    ax.set_title("Mask")

    ax=axs[2]
    cm = plt.imshow(img[..., ::-1] * mask[..., None])
    plt.colorbar(cm, ax=ax)
    ax.set_title("Masked Img")
    
    
    xmin = np.where(np.sum(mask, 0))[0][0]
    xmax = np.where(np.sum(mask, 0))[0][-1]
    
    ymin = np.where(np.sum(mask, 1))[0][0]
    ymax = np.where(np.sum(mask, 1))[0][-1]
    
    ax.set_xlim([xmin-35, xmax+35])
    ax.set_ylim([ymax+35, ymin-35])
    
    if bbox is not None:
        bbox_plot(ax=ax, bbox=bbox)
    
    if supertitle:
        plt.suptitle(supertitle)

    plt.show()

### Make a folder and copy files for clipdrop

In [ ]:
if exists(clip_dir):
    shutil.rmtree(clip_dir)
os.makedirs(clip_dir, exist_ok=True)
os.makedirs(clip_out_dir, exist_ok=True)

cams = ('N1', )
degs = tuple(3*i for i in range(120))
degs = degs[1::2]
degs = degs + (30, )

for name in tqdm(names):
    in_dir = join(raw_dir, name, name)

    
    for cam in cams:
        for deg in degs:
            img = read_img(in_dir, cam=cam, deg=deg)
            assert img is not None

            # depth/mask.png
            mask_old = read_mask(in_dir, cam=cam, deg=deg)
            
            bbox = bbox_from_mask(mask_old)

            cut = bbox_cut_image(bbox, img)

            fname_out = join(clip_dir, f"InfECCVimg_{name}_{cam}_{deg:03d}.jpg")
            assert cv2.imwrite(fname_out, cut)
            
            with open(fname_out.replace(".jpg", ".pkl"), "wb") as fp:
                pickle.dump(file=fp, obj=bbox)

### MANUAL PROCESSING REQUIRED

Now you have to upload the images from `clip_dir` to clipdrop and process them there. With a pro account one can upload 100 images at a  time and use multiple tabs simultaneously. **For clipdrop change the options such that (a) the mask is included in the alpha channel and (b) the image is not cropped.**

After clipdrop is completely finished the next cell will move the images from your download folder.

In [ ]:
# After download
files = sorted(glob(join(ddir, "InfECCVimg_*.png")))
print(len(files))

for file in tqdm(files):
    bname = basename(file)
    shutil.move(file, join(clip_out_dir, bname))

### Write final output



In [ ]:
os.makedirs(proc_dir, exist_ok=True)
for name in tqdm(names):
    in_dir = join(raw_dir, name, name)
    out_dir = join(proc_dir, name)
    os.makedirs(out_dir, exist_ok=True)
    
    shutil.copyfile(src=join(in_dir, "meshes", "poisson.ply"), dst=join(out_dir, "mesh_world.ply"))

    for cam in cams:
        for deg in degs:
            img = read_img(in_dir, cam=cam, deg=deg)
            assert img is not None

            # depth/mask.png
            mask_old = read_mask(in_dir, cam=cam, deg=deg)
            
            bbox = bbox_from_mask(mask_old)

            cut = bbox_cut_image(bbox, img)

            fname_out = join(clip_dir, f"InfECCVimg_{name}_{cam}_{deg:03d}.jpg")
            
            with open(fname_out.replace(".jpg", ".pkl"), "rb") as fp:
                bbox_load = pickle.load(fp)
            
            assert bbox == bbox_load
            
            cut_masked = cv2.imread(join(clip_out_dir, f"InfECCVimg_{name}_{cam}_{deg:03d}.png"), -1)
            assert cut_masked.shape[-1] == 4  # BGRA
            
            view_dir = join(out_dir, f"{cam}_{deg:03d}")
            os.makedirs(view_dir, exist_ok=True)
            os.makedirs(join(view_dir, "depth"), exist_ok=True)
            os.makedirs(join(view_dir, "image"), exist_ok=True)

            # depth/cameras.npz
            cam_to_world = transformation_cam_to_world(in_dir, cam=cam, deg=deg, t="rgb")
            intrinsics = np.pad(read_K(in_dir, cam=cam), [(0, 0), (0, 1)])
            np.savez(join(view_dir, "depth", "cameras.npz"), **{"world_mat_0": cam_to_world[:3, :4], "camera_mat_0": intrinsics})

            # image/000.png
            assert cv2.imwrite(join(view_dir, "image", "000.png"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
            
            mask_new = np.zeros_like(mask_old)
            mask_new[bbox.y[0]:bbox.y[1], bbox.x[0]:bbox.x[1]] = cut_masked[..., -1] > 0

            # depth/mask.png
            assert cv2.imwrite(join(view_dir, "depth", "mask.old.png"), 255*mask_old)
            assert cv2.imwrite(join(view_dir, "depth", "mask.png"), 255*mask_new)
            
            del mask_new
            
    all_degs = tuple(3*i for i in range(120))
    
    train_cams = cams
    visualize_cams = cams
    
    train_degs = all_degs[1::2]
    visualize_degs = (30, )
    
    lines = []
    for cam in train_cams:
        for deg in train_degs:
            lines.append(f"{cam}_{deg:03d}\n")
    print(f"Num train: {len(lines)}")
    with open(join(out_dir, "train.lst"), "w") as fp:
        fp.write("".join(lines))


    lines = []
    for cam in visualize_cams:
        for deg in visualize_degs:
            lines.append(f"{cam}_{deg:03d}\n")
    print(f"Num val/test/vis: {len(lines)}")
    for split in ("test", "val", "visualize"):
        with open(join(out_dir, f"{split}.lst"), "w") as fp:
            fp.write("".join(lines))